<h1> Project 1: Discount Cash Flow Model </h1>

<p>The outline of this project is to request information on stock X from yfinance library and filtering the data for the company's financial statements.

Then utilising this information to calculate DCF.</p>

<p>Notes:

- By utilising yfinance, this is very simplified and does not go into calculating FCF through the financial statements.
- I have made very strong assumptions on all the rates used.</p>

<h2> Perpetual Growth Model </h2>

<p>Our first DCF model will be based on the company existing forever and growing at a constant, reasonable rate.</p>
<p>We will assume a perpetual growth rate for 3 different kinds of growth, looking at the premium/discount of each growth rates.</p>

In [ ]:
# Importing required libraries
import yfinance as yf
import pandas as pd

<p>Below, you can input the stock ticker of choice and we pull data from yfinance API on each financial statement.</p>

<p>Using .loc and .iloc, we can find recent and specific data required for calculating DCF value.</p>

In [171]:
symbol = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, GOOG): ").upper()

ticker = yf.Ticker(symbol)

df_cashflow = pd.DataFrame(ticker.cash_flow) # Yahoo Finance data on the company's cash flow
df_income = pd.DataFrame(ticker.income_stmt) # Data on the company's income statement
df_balance = pd.DataFrame(ticker.balance_sheet) # Data on the company's balance sheet


recent_fcf = df_cashflow.loc['Free Cash Flow'].iloc[0] # Will be used as our exit FCF for DCF calculation

sharesOutstanding = ticker.info['sharesOutstanding'] # Number of shares outstanding for the company
closing_price = ticker.info['previousClose'] # Closing price of the stock
market_cap = sharesOutstanding * closing_price # Market Capitalization of the company


debt = df_balance.loc['Total Debt'].fillna(0).iloc[0]  # Total Debt from the balance sheet


cash = df_balance.loc['Cash And Cash Equivalents'].fillna(0).iloc[0]  + df_balance.loc['Other Short Term Investments'].fillna(0).iloc[0] # Cash from the balance sheet

try:
    interest_expense = df_income.loc['Interest Expense'].fillna(0).iloc[0] # Interest Expense from the income statement
except KeyError:
    interest_expense = 0

<h3>Assumptions</h3>

<p>Used estimates for growth rates, corporate tax rates, risk-free rates, and market return.</p>

In [172]:
low_perp_growth = 0.02  # 2% perpetual growth rate
base_perp_growth = 0.03  # 3% perpetual growth rate
high_perp_growth = 0.05  # 5% perpetual growth rate

tax_rate = 0.20  # 20% corporate tax rate
risk_free_rate = 0.02  # 2% risk-free rate
market_return = 0.1  # 10% expected market return
beta = ticker.info['beta']  # Beta of the stock

cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate) # CAPM formula to calculate cost of equity

if interest_expense != 0 and debt != 0:
    cost_of_debt = (interest_expense / debt) * (1 - tax_rate)  # After-tax cost of debt
else:
    cost_of_debt = 0

if cost_of_debt != 0:
    WACC = (cost_of_equity * (market_cap / (market_cap + debt))) + (cost_of_debt * (debt / (market_cap + debt)))  # Weighted Average Cost of Capital used as discount rate
else:
    WACC = (cost_of_equity * (market_cap / (market_cap + debt)))

<h3>Calculations and Conculsion</h3>

<p>The final section of code is to find the present and intrisic value for each growth rate, then displaying all the information, assumptions, intrinsic value, and premium/discount for the stock.</p>

In [173]:
present_value_low = recent_fcf * (1 + low_perp_growth) / (WACC - low_perp_growth)
present_value_base = recent_fcf * (1 + base_perp_growth) / (WACC - base_perp_growth)
present_value_high = recent_fcf * (1 + high_perp_growth) / (WACC - high_perp_growth)

intrinsic_value_per_share_low = (present_value_low + cash - debt)/ sharesOutstanding
intrinsic_value_per_share_base = (present_value_base + cash - debt)/ sharesOutstanding
intrinsic_value_per_share_high = (present_value_high + cash - debt)/ sharesOutstanding

print(f"Company Name: {ticker.info['longName']}")
print(f"Ticker Symbol: {symbol}")
print(f"The current market price of {symbol} is: ${closing_price:.2f}")
print(f"Number of Shares Outstanding: {sharesOutstanding:,}")
print(f"Total Debt: ${debt:,.2f}")
print(f"Total Cash: ${cash:,.2f}")
print(f"Interest Expense: ${interest_expense:,.2f}", end="\n\n")

print("Assumptions:")
print(f"Perpetual Growth Rates: Low = {low_perp_growth*100}%, Base = {base_perp_growth*100}%, High = {high_perp_growth*100}%")
print(f"Corporate Tax Rate: {tax_rate*100}%")
print(f"Risk-Free Rate: {risk_free_rate*100}%")
print(f"Market Return: {market_return*100}%")
print(f"Beta: {beta:.2f}")
print(f"Cost of Equity: {cost_of_equity*100:.2f}%")
print(f"Cost of Debt: {cost_of_debt*100:.2f}%")
print(f"WACC (Discount Rate): {WACC*100:.2f}%", end="\n\n")

print(f"Intrinsic Value per Share (Low Growth): ${intrinsic_value_per_share_low:.2f}")
print(f"Intrinsic Value per Share (Base Growth): ${intrinsic_value_per_share_base:.2f}")
print(f"Intrinsic Value per Share (High Growth): ${intrinsic_value_per_share_high:.2f}", end="\n\n")

print(f"The premium/discount compared to market price (Low Growth): {((intrinsic_value_per_share_low - closing_price) / closing_price) * 100:.2f}% ")
print(f"The premium/discount compared to market price (Base Growth): {((intrinsic_value_per_share_base - closing_price) / closing_price) * 100:.2f}% ")
print(f"The premium/discount compared to market price (High Growth): {((intrinsic_value_per_share_high - closing_price) / closing_price) * 100:.2f}% ")


Company Name: Catalyst Pharmaceuticals, Inc.
Ticker Symbol: CPRX
The current market price of CPRX is: $23.02
Number of Shares Outstanding: 122,912,387
Total Debt: $3,188,000.00
Total Cash: $517,553,000.00
Interest Expense: $0.00

Assumptions:
Perpetual Growth Rates: Low = 2.0%, Base = 3.0%, High = 5.0%
Corporate Tax Rate: 20.0%
Risk-Free Rate: 2.0%
Market Return: 10.0%
Beta: 0.69
Cost of Equity: 7.50%
Cost of Debt: 0.00%
WACC (Discount Rate): 7.50%

Intrinsic Value per Share (Low Growth): $40.31
Intrinsic Value per Share (Base Growth): $48.78
Intrinsic Value per Share (High Growth): $86.08

The premium/discount compared to market price (Low Growth): 75.12% 
The premium/discount compared to market price (Base Growth): 111.91% 
The premium/discount compared to market price (High Growth): 273.96% 


<h2>Gordon Growth Model</h2>

<p>This DCF model assumes that the company will exit the market after 20 years.</p>